# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [37]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [38]:
# checking current working directory
print(os.getcwd())

# Geting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-17-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-24-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [39]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [40]:
# checking the number of rows in the created csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [41]:
# Making a connection to Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [42]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    )

except Exception as e:
        print(e)

#### Set Keyspace

In [43]:
# Setting 'events' KEYSPACE  so as to be able to execute queries

try:
    session.set_keyspace('events')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [44]:
# Create table for Query 1 below #
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during #
# sessionId = 338, and itemInSession = 4 #
# select artist_name, song_title, length from music_library WHERE session_id = 338 AND  item_in_session = 4 #

query = "CREATE TABLE IF NOT EXISTS music_library "
query = query + "(session_id int, item_in_session int, artist_name text, song_title text, length float, PRIMARY KEY(session_id, item_in_session))"

## Primary key fields are session_id and item_in_session . If we look in the query WHERE clause, partition key is session_id and our clustering column is item_in_session

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [45]:
# Reading event_data_new.csv file and inserting field values for the music_library table #
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Assigning INSERT statements into the `query` variable
        query = "INSERT INTO music_library (session_id, item_in_session, artist_name, song_title, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## Assigning column elements to each column in the INSERT statement and executing the INSERT query
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [46]:
##  SELECT statement to execute query 1 and verify that data was entered in the table
query = "select artist_name, song_title, length from music_library WHERE session_id = 338 AND  item_in_session = 4"

try:
    rows = session.execute(query)
    ## convert resultset to Pandas DataFrame
    df = pd.DataFrame(list(rows))
    
except Exception as e:
    print(e)

df

,artist_name,song_title,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### LET'S REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [47]:
## Creating music_sessions table for the query below
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
# select artist_name, song_title, first_name, last_name from music_sessions WHERE user_id = 10 AND  session_id = 182 #


query = "CREATE TABLE IF NOT EXISTS music_sessions "
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_title text, first_name text, last_name text,  PRIMARY KEY(user_id, session_id, item_in_session))"

## Primary Key fields are user_id, session_id and item_in_session. If we look in the query WHERE clause, user_id is required as a partition key while session_id and item_in_session are clustering columns 


try:
    session.execute(query)
except Exception as e:
    print(e)
                    
                    

In [48]:
# Reading event_data_new.csv file and inserting field values for the music_sessions table #
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Assigning INSERT statements into the `query` variable
        query = "INSERT INTO music_sessions (user_id, session_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s )"
        ## Assigning column elements to each column in the INSERT statement and executing the INSERT query
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4] ))

In [49]:
##  SELECT statement to execute query 2 and verify that data was entered in the table
query = "select artist_name, song_title, first_name, last_name from music_sessions WHERE user_id = 10 AND  session_id = 182"

try:
    
    rows = session.execute(query)
    ## convert resultset to Pandas DataFrame
    df = pd.DataFrame(list(rows))

except Exception as e:
    print(e)

df

,artist_name,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [50]:
## Creating table for query 3 below
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# select first_name, last_name from song_library WHERE song_title = 'All Hands Against His Own #

query = "CREATE TABLE IF NOT EXISTS song_library "
query = query + "(song_title text, session_id int, item_in_session int,  first_name text, last_name text,  PRIMARY KEY(song_title, first_name, last_name))"
## Primary key fields are  song_title, session_id and item_in_session fields. If we look in the query WHERE clause, partition key is song_title and the other 2 fields are clustering columns

try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [51]:
# Setting up the CSV file and inserting csv row values into song_library table#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
##  Assigning INSERT statements into the `query` variable
        query = "INSERT INTO song_library (song_title, session_id, item_in_session, first_name, last_name)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## Assigning column elements to each column in the INSERT statement and executing the INSERT query
        session.execute(query, (line[9], int(line[8]), int(line[3]), line[1], line[4] ))

In [52]:
## SELECT statement to execute query 3 and verify that data was entered in the table
query = "select  first_name, last_name from song_library WHERE song_title = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
    ## convert resultset to Pandas DataFrame
    df = pd.DataFrame(list(rows))

except Exception as e:
    print(e)

df

,first_name,last_name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [53]:
###  Let's drop the table before closing out the sessions or use the drop table statements to reset our database and test our ETL pipeline

In [54]:
query = "drop table music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table music_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [55]:
session.shutdown()
cluster.shutdown()